In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np
from xgboost import XGBRegressor

In [2]:
df_09_10 = pd.read_csv("Year 2009-2010.csv",encoding ='ISO-8859-1')
df_10_11 = pd.read_csv("Year 2010-2011.csv",encoding ='ISO-8859-1')

In [3]:
df_09_10 = df_09_10[['InvoiceDate','Invoice', 'Customer ID', 'StockCode', 'Description', 'Quantity',
       'Price', 'Country']]
df_10_11 = df_10_11[['InvoiceDate','Invoice', 'Customer ID', 'StockCode', 'Description', 'Quantity',
       'Price', 'Country']]

In [4]:
df_09_10['InvoiceDate'] = pd.to_datetime(df_09_10['InvoiceDate'])
df_10_11['InvoiceDate'] = pd.to_datetime(df_10_11['InvoiceDate'])

In [5]:
df_09_10.shape

(525461, 8)

In [6]:
df_09_10 = df_09_10[~df_09_10['Invoice'].astype(str).str.startswith("C")]
df_10_11 = df_10_11[~df_10_11['Invoice'].astype(str).str.startswith("C")]

In [7]:
df_09_10 = df_09_10[(df_09_10["Price"]>0) & (df_09_10["Quantity"]>0)]
df_10_11 = df_10_11[(df_10_11["Price"]>0) & (df_10_11["Quantity"]>0)]

In [8]:
df_09_10 = df_09_10[df_09_10["StockCode"].astype(str).str.isnumeric()]
df_10_11 = df_10_11[df_10_11["StockCode"].astype(str).str.isnumeric()]

In [9]:
# df_09_10["Customer ID"] = df_09_10["Customer ID"].astype(int)
# # .astype(str).str.isnumeric().value_counts()

In [10]:
df_09_10.isnull().sum()

InvoiceDate        0
Invoice            0
Customer ID    83980
StockCode          0
Description        0
Quantity           0
Price              0
Country            0
dtype: int64

In [11]:
df_09_10[["Quantity", "Price"]].describe()

,Quantity,Price
count,434882.000000,434882.000000
mean,11.631769,3.385278
std,90.222332,5.235174
min,1.000000,0.030000
25%,1.000000,1.250000
50%,4.000000,2.100000
75%,12.000000,4.210000
max,19152.000000,1157.150000


In [12]:
df = pd.concat([df_09_10,df_10_11],ignore_index=True)

In [13]:
df.head()

,InvoiceDate,Invoice,Customer ID,StockCode,Description,Quantity,Price,Country
0,2009-12-01 07:45:00,489434,13085.0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,United Kingdom
1,2009-12-01 07:45:00,489434,13085.0,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,United Kingdom
2,2009-12-01 07:45:00,489434,13085.0,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,United Kingdom
3,2009-12-01 07:45:00,489434,13085.0,22064,PINK DOUGHNUT TRINKET POT,24,1.65,United Kingdom
4,2009-12-01 07:45:00,489434,13085.0,21871,SAVE THE PLANET MUG,24,1.25,United Kingdom


In [14]:
df.shape

(911978, 8)

In [15]:
df.head(5)

,InvoiceDate,Invoice,Customer ID,StockCode,Description,Quantity,Price,Country
0,2009-12-01 07:45:00,489434,13085.0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,United Kingdom
1,2009-12-01 07:45:00,489434,13085.0,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,United Kingdom
2,2009-12-01 07:45:00,489434,13085.0,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,United Kingdom
3,2009-12-01 07:45:00,489434,13085.0,22064,PINK DOUGHNUT TRINKET POT,24,1.65,United Kingdom
4,2009-12-01 07:45:00,489434,13085.0,21871,SAVE THE PLANET MUG,24,1.25,United Kingdom


In [16]:
df = df.sort_values('InvoiceDate').reset_index(drop=True)

In [17]:
df

,InvoiceDate,Invoice,Customer ID,StockCode,Description,Quantity,Price,Country
0,2009-12-01 07:45:00,489434,13085.0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,United Kingdom
1,2009-12-01 07:45:00,489434,13085.0,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,United Kingdom
2,2009-12-01 07:45:00,489434,13085.0,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,United Kingdom
3,2009-12-01 07:45:00,489434,13085.0,22064,PINK DOUGHNUT TRINKET POT,24,1.65,United Kingdom
4,2009-12-01 07:45:00,489434,13085.0,21871,SAVE THE PLANET MUG,24,1.25,United Kingdom
...,...,...,...,...,...,...,...,...
911973,2011-12-09 12:50:00,581587,12680.0,22555,PLASTERS IN TIN STRONGMAN,12,1.65,France
911974,2011-12-09 12:50:00,581587,12680.0,22556,PLASTERS IN TIN CIRCUS PARADE,12,1.65,France
911975,2011-12-09 12:50:00,581587,12680.0,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,France
911976,2011-12-09 12:50:00,581587,12680.0,22730,ALARM CLOCK BAKELIKE IVORY,4,3.75,France


In [18]:
df["InvoiceDate"].min(), df["InvoiceDate"].max()

(Timestamp('2009-12-01 07:45:00'), Timestamp('2011-12-09 12:50:00'))

In [19]:
df[["Quantity", "Price"]].min()

Quantity    1.00
Price       0.03
dtype: float64

In [20]:
df['date'] = df['InvoiceDate'].dt.date

In [21]:
df

,InvoiceDate,Invoice,Customer ID,StockCode,Description,Quantity,Price,Country,date
0,2009-12-01 07:45:00,489434,13085.0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,United Kingdom,2009-12-01
1,2009-12-01 07:45:00,489434,13085.0,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,United Kingdom,2009-12-01
2,2009-12-01 07:45:00,489434,13085.0,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,United Kingdom,2009-12-01
3,2009-12-01 07:45:00,489434,13085.0,22064,PINK DOUGHNUT TRINKET POT,24,1.65,United Kingdom,2009-12-01
4,2009-12-01 07:45:00,489434,13085.0,21871,SAVE THE PLANET MUG,24,1.25,United Kingdom,2009-12-01
...,...,...,...,...,...,...,...,...,...
911973,2011-12-09 12:50:00,581587,12680.0,22555,PLASTERS IN TIN STRONGMAN,12,1.65,France,2011-12-09
911974,2011-12-09 12:50:00,581587,12680.0,22556,PLASTERS IN TIN CIRCUS PARADE,12,1.65,France,2011-12-09
911975,2011-12-09 12:50:00,581587,12680.0,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,France,2011-12-09
911976,2011-12-09 12:50:00,581587,12680.0,22730,ALARM CLOCK BAKELIKE IVORY,4,3.75,France,2011-12-09


In [22]:
daily_demand = (df.groupby(['StockCode','date'])['Quantity'].sum().reset_index())

In [23]:
daily_demand["date"] = pd.to_datetime(daily_demand["date"])

In [24]:
daily_demand

,StockCode,date,Quantity
0,10002,2009-12-01,12
1,10002,2009-12-03,7
2,10002,2009-12-04,73
3,10002,2009-12-06,49
4,10002,2009-12-07,2
...,...,...,...
453528,90208,2011-11-28,1
453529,90208,2011-11-30,1
453530,90208,2011-12-06,1
453531,90208,2011-12-08,6


In [25]:
daily_demand = (
    daily_demand
    .set_index("date")
    .groupby("StockCode")["Quantity"]
    .apply(lambda x: x.asfreq("D", fill_value=0))
    .reset_index()
)

In [26]:
daily_demand

,StockCode,date,Quantity
0,10002,2009-12-01,12
1,10002,2009-12-02,0
2,10002,2009-12-03,7
3,10002,2009-12-04,73
4,10002,2009-12-05,0
...,...,...,...
1591340,90208,2011-12-05,0
1591341,90208,2011-12-06,1
1591342,90208,2011-12-07,0
1591343,90208,2011-12-08,6


In [27]:
sku_total = daily_demand.groupby('StockCode')[['Quantity']].sum()
sku_total

,Quantity
StockCode,
10002,8836
10080,315
10109,4
10120,680
10125,2151
...,...
90192,97
90194,33
90203,73


In [28]:
valid_sku = sku_total[sku_total["Quantity"]>50].index

In [29]:
daily_demand = daily_demand[daily_demand['StockCode'].isin(valid_sku)]

In [30]:
daily_demand

,StockCode,date,Quantity
0,10002,2009-12-01,12
1,10002,2009-12-02,0
2,10002,2009-12-03,7
3,10002,2009-12-04,73
4,10002,2009-12-05,0
...,...,...,...
1591340,90208,2011-12-05,0
1591341,90208,2011-12-06,1
1591342,90208,2011-12-07,0
1591343,90208,2011-12-08,6


In [31]:
daily_demand["lag_1"] = daily_demand.groupby("StockCode")["Quantity"].shift(1)
daily_demand["lag_7"] = daily_demand.groupby("StockCode")["Quantity"].shift(7)
daily_demand["lag_14"] = daily_demand.groupby("StockCode")["Quantity"].shift(14)

/var/folders/dg/19j8hs653295z30t_h9w01mm0000gn/T/ipykernel_9128/2768138918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_demand["lag_1"] = daily_demand.groupby("StockCode")["Quantity"].shift(1)
/var/folders/dg/19j8hs653295z30t_h9w01mm0000gn/T/ipykernel_9128/2768138918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_demand["lag_7"] = daily_demand.groupby("StockCode")["Quantity"].shift(7)
/var/folders/dg/19j8hs653295z30t_h9w01mm0000gn/T/ipykernel_9128/2768138918.py:3: SettingWithCopyWarn

In [32]:
daily_demand

,StockCode,date,Quantity,lag_1,lag_7,lag_14
0,10002,2009-12-01,12,NaN,NaN,NaN
1,10002,2009-12-02,0,12.0,NaN,NaN
2,10002,2009-12-03,7,0.0,NaN,NaN
3,10002,2009-12-04,73,7.0,NaN,NaN
4,10002,2009-12-05,0,73.0,NaN,NaN
...,...,...,...,...,...,...
1591340,90208,2011-12-05,0,0.0,1.0,1.0
1591341,90208,2011-12-06,1,0.0,0.0,0.0
1591342,90208,2011-12-07,0,1.0,1.0,0.0
1591343,90208,2011-12-08,6,0.0,0.0,1.0


In [33]:
daily_demand["roll_mean_7"] = (
    daily_demand.groupby("StockCode")["Quantity"]
    .shift(1)
    .rolling(7)
    .mean()
)

daily_demand["roll_std_7"] = (
    daily_demand.groupby("StockCode")["Quantity"]
    .shift(1)
    .rolling(7)
    .std()
)

/var/folders/dg/19j8hs653295z30t_h9w01mm0000gn/T/ipykernel_9128/2354129574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_demand["roll_mean_7"] = (
/var/folders/dg/19j8hs653295z30t_h9w01mm0000gn/T/ipykernel_9128/2354129574.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_demand["roll_std_7"] = (


In [34]:
daily_demand

,StockCode,date,Quantity,lag_1,lag_7,lag_14,roll_mean_7,roll_std_7
0,10002,2009-12-01,12,NaN,NaN,NaN,NaN,NaN
1,10002,2009-12-02,0,12.0,NaN,NaN,NaN,NaN
2,10002,2009-12-03,7,0.0,NaN,NaN,NaN,NaN
3,10002,2009-12-04,73,7.0,NaN,NaN,NaN,NaN
4,10002,2009-12-05,0,73.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1591340,90208,2011-12-05,0,0.0,1.0,1.0,0.285714,0.487950
1591341,90208,2011-12-06,1,0.0,0.0,0.0,0.142857,0.377964
1591342,90208,2011-12-07,0,1.0,1.0,0.0,0.285714,0.487950
1591343,90208,2011-12-08,6,0.0,0.0,1.0,0.142857,0.377964


In [35]:
daily_demand["day_of_week"] = daily_demand["date"].dt.weekday
daily_demand["month"] = daily_demand["date"].dt.month

/var/folders/dg/19j8hs653295z30t_h9w01mm0000gn/T/ipykernel_9128/4273600296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_demand["day_of_week"] = daily_demand["date"].dt.weekday
/var/folders/dg/19j8hs653295z30t_h9w01mm0000gn/T/ipykernel_9128/4273600296.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_demand["month"] = daily_demand["date"].dt.month


In [36]:
daily_demand

,StockCode,date,Quantity,lag_1,lag_7,lag_14,roll_mean_7,roll_std_7,day_of_week,month
0,10002,2009-12-01,12,NaN,NaN,NaN,NaN,NaN,1,12
1,10002,2009-12-02,0,12.0,NaN,NaN,NaN,NaN,2,12
2,10002,2009-12-03,7,0.0,NaN,NaN,NaN,NaN,3,12
3,10002,2009-12-04,73,7.0,NaN,NaN,NaN,NaN,4,12
4,10002,2009-12-05,0,73.0,NaN,NaN,NaN,NaN,5,12
...,...,...,...,...,...,...,...,...,...,...
1591340,90208,2011-12-05,0,0.0,1.0,1.0,0.285714,0.487950,0,12
1591341,90208,2011-12-06,1,0.0,0.0,0.0,0.142857,0.377964,1,12
1591342,90208,2011-12-07,0,1.0,1.0,0.0,0.285714,0.487950,2,12
1591343,90208,2011-12-08,6,0.0,0.0,1.0,0.142857,0.377964,3,12


In [37]:
daily_demand = daily_demand.dropna()

In [38]:
daily_demand

,StockCode,date,Quantity,lag_1,lag_7,lag_14,roll_mean_7,roll_std_7,day_of_week,month
14,10002,2009-12-15,0,37.0,12.0,12.0,8.428571,13.525989,1,12
15,10002,2009-12-16,0,0.0,0.0,0.0,6.714286,13.756384,2,12
16,10002,2009-12-17,0,0.0,1.0,7.0,6.714286,13.756384,3,12
17,10002,2009-12-18,0,0.0,9.0,73.0,6.571429,13.830608,4,12
18,10002,2009-12-19,0,0.0,0.0,0.0,5.285714,13.984686,5,12
...,...,...,...,...,...,...,...,...,...,...
1591340,90208,2011-12-05,0,0.0,1.0,1.0,0.285714,0.487950,0,12
1591341,90208,2011-12-06,1,0.0,0.0,0.0,0.142857,0.377964,1,12
1591342,90208,2011-12-07,0,1.0,1.0,0.0,0.285714,0.487950,2,12
1591343,90208,2011-12-08,6,0.0,0.0,1.0,0.142857,0.377964,3,12


In [39]:
train = daily_demand[daily_demand["date"] < "2011-07-01"]
test  = daily_demand[daily_demand["date"] >= "2011-07-01"]

In [40]:
train

,StockCode,date,Quantity,lag_1,lag_7,lag_14,roll_mean_7,roll_std_7,day_of_week,month
14,10002,2009-12-15,0,37.0,12.0,12.0,8.428571,13.525989,1,12
15,10002,2009-12-16,0,0.0,0.0,0.0,6.714286,13.756384,2,12
16,10002,2009-12-17,0,0.0,1.0,7.0,6.714286,13.756384,3,12
17,10002,2009-12-18,0,0.0,9.0,73.0,6.571429,13.830608,4,12
18,10002,2009-12-19,0,0.0,0.0,0.0,5.285714,13.984686,5,12
...,...,...,...,...,...,...,...,...,...,...
1591178,90208,2011-06-26,0,0.0,0.0,0.0,0.000000,0.000000,6,6
1591179,90208,2011-06-27,0,0.0,0.0,0.0,0.000000,0.000000,0,6
1591180,90208,2011-06-28,0,0.0,0.0,0.0,0.000000,0.000000,1,6
1591181,90208,2011-06-29,1,0.0,0.0,0.0,0.000000,0.000000,2,6


In [41]:
test

,StockCode,date,Quantity,lag_1,lag_7,lag_14,roll_mean_7,roll_std_7,day_of_week,month
1080,10080,2011-07-01,0,12.0,0.0,0.0,1.714286,4.535574,4,7
1081,10080,2011-07-02,0,0.0,0.0,0.0,1.714286,4.535574,5,7
1082,10080,2011-07-03,0,0.0,0.0,0.0,1.714286,4.535574,6,7
1083,10080,2011-07-04,0,0.0,0.0,0.0,1.714286,4.535574,0,7
1084,10080,2011-07-05,12,0.0,0.0,48.0,1.714286,4.535574,1,7
...,...,...,...,...,...,...,...,...,...,...
1591340,90208,2011-12-05,0,0.0,1.0,1.0,0.285714,0.487950,0,12
1591341,90208,2011-12-06,1,0.0,0.0,0.0,0.142857,0.377964,1,12
1591342,90208,2011-12-07,0,1.0,1.0,0.0,0.285714,0.487950,2,12
1591343,90208,2011-12-08,6,0.0,0.0,1.0,0.142857,0.377964,3,12


In [42]:
features = [
    "lag_1", "lag_7", "lag_14",
    "roll_mean_7", "roll_std_7",
    "day_of_week", "month"
]

target = "Quantity"


In [43]:

model = XGBRegressor(
    n_estimators=200,
    learning_rate=0.04,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(train[features], train[target])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.04, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)

In [44]:

preds = model.predict(test[features])
rmse = np.sqrt(mean_squared_error(test[target], preds))

print("RMSE:", rmse)

RMSE: 35.55947243267815


In [45]:
baseline_rmse = np.sqrt(mean_squared_error(
    test["Quantity"],
    test["lag_1"]
))

In [46]:
baseline_rmse

49.42897700311046

In [47]:
def forecast_21_days(model, sku_df, features):
    sku_df = sku_df.sort_values("date")
    history = sku_df.copy()

    forecasts = []

    for step in range(1, 22):
        last_row = history.iloc[-1]

        row = {
            "lag_1": last_row["Quantity"],
            "lag_7": history.iloc[-7]["Quantity"] if len(history) >= 7 else last_row["Quantity"],
            "lag_14": history.iloc[-14]["Quantity"] if len(history) >= 14 else last_row["Quantity"],
            "roll_mean_7": history["Quantity"].tail(7).mean(),
            "roll_std_7": history["Quantity"].tail(7).std(),
            "day_of_week": (last_row["date"] + pd.Timedelta(days=1)).weekday(),
            "month": (last_row["date"] + pd.Timedelta(days=1)).month
        }

        X = pd.DataFrame([row])[features]
        y_pred = model.predict(X)[0]

        forecasts.append(max(0, y_pred))

        history = pd.concat([
            history,
            pd.DataFrame({
                "date": [last_row["date"] + pd.Timedelta(days=1)],
                "Quantity": [y_pred]
            })
        ], ignore_index=True)

    return forecasts

In [48]:
results = []

for sku, sku_df in daily_demand.groupby("StockCode"):
    preds = forecast_21_days(model, sku_df, features)

    row = {"StockCode": sku}
    for i, val in enumerate(preds, 1):
        row[f"Day_{i}"] = val

    results.append(row)

forecast_df = pd.DataFrame(results)

In [49]:
forecast_df.to_csv("21_day_forecast.csv", index=False)

In [50]:
forecast_df.describe()

,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,...,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21
count,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,...,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000
mean,4.377113,5.300149,9.120026,11.101282,11.683107,12.787986,11.965590,6.052203,7.107751,11.164748,...,14.864125,16.345942,14.860451,8.137281,8.890527,13.705883,16.666301,17.796298,19.651549,18.039593
std,7.873747,9.076033,20.048606,20.960056,22.151237,24.003009,18.353527,10.981053,12.504755,21.082077,...,29.402927,29.730018,21.785854,14.412046,14.557883,25.407273,31.982128,33.176463,34.415034,27.844219
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.290173,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.307905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.329757
25%,0.362636,0.472969,1.184856,1.770029,1.738571,2.058325,2.769066,0.638101,0.610395,1.754596,...,2.164147,2.566819,4.155478,0.794609,0.767321,1.961325,2.395340,2.974781,4.127663,5.233538
50%,1.788110,2.096128,3.179316,3.995334,4.397085,5.355377,6.249465,3.054320,3.353710,4.680188,...,6.054016,7.537201,7.649323,4.869980,4.888576,5.994694,7.385604,7.852220,8.783820,8.369150
75%,5.056369,6.450351,9.383158,10.804491,11.235373,12.730752,12.716804,7.005630,8.336914,10.346148,...,11.894433,13.376749,14.308105,8.576901,10.026577,11.791312,12.397872,13.580743,15.297076,15.429415
max,140.179291,124.425140,608.493896,274.568848,297.222595,436.885010,294.682251,167.704727,222.066681,280.557343,...,453.984894,388.785004,246.563751,225.352371,145.129425,285.274536,458.080292,451.918549,439.855499,319.628082
